<a href="https://colab.research.google.com/github/asifahsaan/Deep-Learning/blob/main/Image_Preprocessing_and_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Image Preprocessing & Augmentation Notebook
This notebook demonstrates comprehensive image preprocessing techniques used in image classification workflows:
- Image loading and normalization
- Resizing and grayscale conversion
- Brightness, contrast, and flipping
- Augmentation pipeline with `tf.image` and `Keras` layers
- Visualization of augmented images

## 📦 Step 1: Import Required Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib


## 🗂️ Step 2: Load the Flower Dataset

In [ ]:
dataset_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir) / 'flower_photos'
class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
print("Classes:", class_names)

## 🖼️ Step 3: Load a Batch of Images

In [ ]:
batch_size = 8
img_height = 128
img_width = 128

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

## 👁️ Step 4: View Original Images

In [ ]:
plt.figure(figsize=(10, 5))
for images, labels in train_ds.take(1):
    for i in range(8):
        ax = plt.subplot(2, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.tight_layout()
plt.show()

## 🔢 Step 5: Normalize Image Data

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
for image_batch, _ in normalized_ds.take(1):
    print("Min pixel:", tf.reduce_min(image_batch).numpy())
    print("Max pixel:", tf.reduce_max(image_batch).numpy())

## 🌑 Step 6: Convert to Grayscale

In [ ]:
def to_grayscale(images):
    return tf.image.rgb_to_grayscale(images)

for images, _ in train_ds.take(1):
    gray_images = to_grayscale(images)
    plt.figure(figsize=(10, 5))
    for i in range(8):
        ax = plt.subplot(2, 4, i + 1)
        plt.imshow(tf.squeeze(gray_images[i]), cmap='gray')
        plt.axis('off')
    plt.suptitle("Grayscale Images")
    plt.tight_layout()
    plt.show()

## 🎨 Step 7: Image Augmentation using `tf.image`

In [ ]:
def augment_image_tf(img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    return img

for images, _ in train_ds.take(1):
    plt.figure(figsize=(10, 5))
    for i in range(8):
        augmented = augment_image_tf(images[i])
        ax = plt.subplot(2, 4, i + 1)
        plt.imshow(tf.cast(augmented, tf.uint8))
        plt.axis('off')
    plt.suptitle("Augmented Images (Brightness, Contrast, Flip)")
    plt.tight_layout()
    plt.show()

## 🧱 Step 8: Image Augmentation using `tf.keras.layers`

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

for images, _ in train_ds.take(1):
    plt.figure(figsize=(10, 5))
    for i in range(8):
        augmented = data_augmentation(images[i:i+1])
        ax = plt.subplot(2, 4, i + 1)
        plt.imshow(augmented[0].numpy().astype('uint8'))
        plt.axis('off')
    plt.suptitle("Augmented Images via Keras Layers")
    plt.tight_layout()
    plt.show()